In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import paho.mqtt.client as mqtt
import time

In [2]:
# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

In [3]:
# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    print(f"Message received: {last_message} on topic {message.topic}")

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")

In [4]:
# Create an MQTT client instance
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

(<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 1)

In [5]:
x_HP = 100
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

# Loop forever to wait for messages
# client.loop_forever()
client.loop_start()

Message sent: x_HP = 100, x_FAN = 0


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

Message received: 20000101T215344	TMP&U	 24.65	 24.72	 26.44	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215345	TMP&U	 24.66	 24.74	 26.47	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215346	TMP&U	 24.67	 24.75	 26.50	Peltier: 0	Fan: 0	 on topic diya06/rx/


Message received: State:	1 on topic diya06/rx/
 on topic diya06/rx/Buffer:	HELLO
Message received: S on topic diya06/rx/
Message received: Command not recognized. on topic diya06/rx/
Message received: State:	-1 on topic diya06/rx/
 on topic diya06/rx/Buffer:	HELLO
Message received: S on topic diya06/rx/
Message received: 20000101T215347	TMP&U	 24.69	 24.77	 26.53	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215348	TMP&U	 24.71	 24.78	 26.56	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215349	TMP&U	 24.71	 24.80	 26.58	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215350	TMP&U	 24.72	 24.81	 26.61	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215351	TMP&U	 24.74	 24.83	 26.64	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215352	TMP&U	 24.75	 24.84	 26.66	Peltier: 0	Fan: 0	 on topic diya06/rx/
Message received: 20000101T215353	TMP&U	 24.76	 24.85	 26.69	Peltier: 0	Fan: 0	 on topic diya06

In [17]:
x_HP = 0
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

Message sent: x_HP = 0, x_FAN = 0


In [18]:
client.loop_stop()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [19]:
client.disconnect()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
# Start the network loop in the background
client.loop_start()

# Wait until we receive the first message or a timeout
timeout = 10  # Set a timeout of 10 seconds
start_time = time.time()

# while last_message is None and (time.time() - start_time) < timeout:
#     # Wait for the message or timeout
#     time.sleep(0.1)  # Avoid busy-waiting

# Control loop
while True:
    # Wait until we receive some sensor data
    if last_sensor_data is not None:
        # Step 1: Perform the calculation (MPC or some other control logic)
        x_HP = compute_control_input(last_sensor_data)
        
        # Step 2: Send the control input back via MQTT
        send_control_input(client, x_HP)
        
        # Reset last_sensor_data to None to wait for the next measurement
        last_sensor_data = None
    
    # Sleep for a short duration before checking again
    time.sleep(1)

# Stop the network loop and disconnect the client when done (not reached in this loop)
client.loop_stop()
client.disconnect()


In [ ]:
# Received message example
message = "20000101T193040 TMP&U 25.44 27.21 33.60 Peltier: 80 Fan: 0"

# Split the message by spaces
parts = message.split()

# Parse each component based on its position in the message
timestamp = parts[0]  # "20000101T193040"
sensor_label = parts[1]  # "TMP&U"
T_1 = float(parts[2])  # 25.44
T_2 = float(parts[3])  # 27.21
T_3 = float(parts[4])  # 33.60

# Parse the 'Peltier' and 'Fan' values by removing their labels
peltier_value = int(parts[6])  # 80
fan_value = int(parts[8])  # 0

# Print out parsed values
print("Timestamp:", timestamp)
print("Sensor Label:", sensor_label)
print("Measurements:", T_1, T_2, T_3)
print("Peltier Value:", peltier_value)
print("Fan Value:", fan_value)
